In [1]:
import numpy as np
wordsList = np.load('wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
SeqLength = 42
counter = 0
for p in wordsList:
    counter = counter+1
    if counter >= SeqLength:                
        break

print("---------------------------------------")
wordVectors = np.load('wordVectors.npy')
print ('Loaded the word vectors!')

Loaded the word list!
---------------------------------------
Loaded the word vectors!


In [51]:
maxSeqLength = 10 #Maximum length of sentence
numDimensions = 300 #Dimensions for each word vector
inputSentence = np.zeros((maxSeqLength), dtype='int32')
inputSentence[0] = wordsList.index("happiness")
inputSentence[1] = wordsList.index("fear")
inputSentence[2] = wordsList.index("anger")
inputSentence[3] = wordsList.index("sadness")
inputSentence[4] = wordsList.index("disgust")
inputSentence[5] = wordsList.index("'")
inputSentence[6] = wordsList.index("!")
#inputSentence[7],inputSentence[8] and inputSentence[9] are going to be 0
print(inputSentence.shape)
print(inputSentence) #Shows the row index for each word

(10,)
[ 9811  1655  3946 13866 18741    57   805     0     0     0]


In [3]:
import tensorflow as tf
with tf.Session() as sess:
    print(tf.nn.embedding_lookup(wordVectors,inputSentence).eval())

[[ 0.039495    1.5316     -0.39991999 -0.20091     1.37839997  0.23264
   0.40810999  0.09424     0.80492997  1.18900001  0.37090001  0.29229999
   0.051458   -0.91470999  0.79913002  0.031613    0.23425999 -0.078071
   0.55264997 -0.094033   -0.067781    0.94492     0.13196    -0.2352
   1.61380005 -0.31953001 -1.30379999 -0.33394     1.0244      0.12196
   1.58580005  1.08249998 -0.15639    -0.38266999 -0.76346999 -0.49954
  -0.52539998 -0.33105999  0.29729    -0.17441    -0.076043   -0.83092999
  -0.47857001 -0.029247   -0.26328999 -0.30849999  0.15287    -0.30546999
   0.15899    -0.30671   ]
 [ 0.53661001 -0.33871999  0.28771999 -0.72583997 -0.041209    0.28498
   0.21664999  0.62408    -0.1204      0.41135001  0.10111    -0.11545
  -0.15651999 -0.39265001  0.66285998  0.71562999  0.42140001 -0.37041
  -0.081016   -0.29010001 -0.28406999  0.30706999  0.035974    0.32001999
   0.45456001 -2.08299994 -0.65890002 -0.081918    1.0431      0.50839001
   2.54329991  0.81831998 -0.11508 

In [4]:
import pandas as pd
file1 = pd.read_csv('text_emotion.csv')

In [5]:
labelledColumn = file1['label']

In [6]:
ll = labelledColumn.replace("worry","sadness")
ll = ll.replace("love","happiness")
ll = ll.replace("fun","happiness")
ll = ll.replace("enthusiasm","happiness")
ll = ll.replace("empty","sadness")
ll = ll.replace("boredom","neutral")
ll = ll.replace("relief","happiness")
ll = ll.replace("hate","anger")

In [7]:
textList1 = file1['text']

In [8]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("\n", " ")
    return re.sub(strip_special_chars, " ", string.lower())

In [9]:
CleanedText = []
for p in textList1:
    cleanedLine = cleanSentences(p)
    CleanedText.append(cleanedLine)

In [10]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [11]:
filtered_sentence = []
for w in CleanedText:
    splitedSentence = w.split()
    filtering_sentence=[]
    for s in splitedSentence:
        if s not in stop_words:
            filtering_sentence.append(s)
    filtered_sentence.append(filtering_sentence)

In [12]:
filtered_sentence[0]

['tiffanylue',
 'know',
 'listenin',
 'bad',
 'habit',
 'earlier',
 'started',
 'freakin',
 'part']

In [15]:
numWords1 = []
count1=0
for pf in filtered_sentence:
    counter1 = len(pf)
    numWords1.append(counter1)
    if(counter1 > 30):
        count1=count1+1
    
print('files read finished')

numFiles1 = len(numWords1)
print('The total number of files is', numFiles1)
print('The total number of words in the files is', sum(numWords1))
print('The average number of words in the files is', sum(numWords1)/len(numWords1))
print(count1)

files read finished
The total number of files is 40000
The total number of words in the files is 315471
The average number of words in the files is 7.886775
1


In [16]:
maxSeqLength = 40

In [17]:
labelPrediction = ["happiness","fear","anger","sadness","surprise","neutral"]

In [20]:
ids = np.zeros((numFiles1, maxSeqLength), dtype='int32')
fileCounter = 0
for pf in filtered_sentence:
    indexCounter = 0
    for word in pf:
        try:
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
        indexCounter = indexCounter + 1
        if indexCounter >= maxSeqLength:
            break
    fileCounter = fileCounter + 1 
    
np.save('Data/idsMatrix1', ids)

In [21]:
ids = np.load('Data/idsMatrix1.npy')

In [22]:
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1,29999)
        labeltxt = ll[num]
        labelindex = wordsList.index(labeltxt)
        arr[i] = ids[num-1:num]
        if labeltxt=="happiness":
            labels.append([1,0,0,0,0,0])
        elif labeltxt=="fear":
            labels.append([0,1,0,0,0,0])
        elif labeltxt=="anger":
            labels.append([0,0,1,0,0,0])
        elif labeltxt=="sadness":
            labels.append([0,0,0,1,0,0])
        elif labeltxt=="surprise":
            labels.append([0,0,0,0,1,0])
        elif labeltxt=="neutral":
            labels.append([0,0,0,0,0,1])
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(30000,39999)
        #labeltxt = file['label'][num]
        labeltxt = ll[num]
        labelindex = wordsList.index(labeltxt)
        arr[i] = ids[num-1:num]
        if labeltxt=="happiness":
            labels.append([1,0,0,0,0,0])
        elif labeltxt=="fear":
            labels.append([0,1,0,0,0,0])
        elif labeltxt=="anger":
            labels.append([0,0,1,0,0,0])
        elif labeltxt=="sadness":
            labels.append([0,0,0,1,0,0])
        elif labeltxt=="surprise":
            labels.append([0,0,0,0,1,0])
        elif labeltxt=="neutral":
            labels.append([0,0,0,0,0,1])
    return arr, labels

In [23]:
batchSize = 24
lstmUnits = 64
numClasses = 6
iterations = 100000

In [24]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

In [25]:
data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)
data.shape

TensorShape([Dimension(24), Dimension(40), Dimension(50)])

In [26]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

In [27]:
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

In [28]:
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [29]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [30]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [31]:
# sess = tf.InteractiveSession()
# saver = tf.train.Saver()
# sess.run(tf.global_variables_initializer())

# for i in range(iterations):
#    #Next Batch of reviews
#    nextBatch, nextBatchLabels = getTrainBatch();
#    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
   
#    #Write summary to Tensorboard
#    if (i % 50 == 0):
#        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
#        writer.add_summary(summary, i)

#    #Save the network every 10,000 training iterations
#    if (i % 10000 == 0 and i != 0):
#        save_path = saver.save(sess, "file1FilteredModels/pretrained_lstm.ckpt", global_step=i)
#        print("saved to %s" % save_path)
# writer.close()

saved to file1FilteredModels/pretrained_lstm.ckpt-10000
saved to file1FilteredModels/pretrained_lstm.ckpt-20000
saved to file1FilteredModels/pretrained_lstm.ckpt-30000
saved to file1FilteredModels/pretrained_lstm.ckpt-40000
saved to file1FilteredModels/pretrained_lstm.ckpt-50000
saved to file1FilteredModels/pretrained_lstm.ckpt-60000
saved to file1FilteredModels/pretrained_lstm.ckpt-70000
saved to file1FilteredModels/pretrained_lstm.ckpt-80000
saved to file1FilteredModels/pretrained_lstm.ckpt-90000


In [32]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('file1FilteredModels'))

INFO:tensorflow:Restoring parameters from file1FilteredModels\pretrained_lstm.ckpt-90000


In [33]:
iterations = 10
for i in range(iterations):
    nextBatch, nextBatchLabels = getTestBatch();
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)

Accuracy for this batch: 41.6666656733
Accuracy for this batch: 25.0
Accuracy for this batch: 20.8333328366
Accuracy for this batch: 37.5
Accuracy for this batch: 20.8333328366
Accuracy for this batch: 20.8333328366
Accuracy for this batch: 25.0
Accuracy for this batch: 16.6666671634
Accuracy for this batch: 50.0
Accuracy for this batch: 33.3333343267


In [35]:
def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unknown words
    return sentenceMatrix

In [48]:
inputText = "freak"
inputMatrix = getSentenceMatrix(inputText)

In [49]:
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]

# print("happiness = ",predictedSentiment[0])
# print("fear=",predictedSentiment[1])
# print("anger=",predictedSentiment[2])
# print("sadness=",predictedSentiment[3])
# print("surprise=",predictedSentiment[4])
# print("neutral=",predictedSentiment[5])

import operator
index, value = max(enumerate(predictedSentiment), key=operator.itemgetter(1))

labelPrediction[index]

'happiness'